In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

In [8]:
data = '/Users/utsav/Desktop/repos/Pandas-Cookbook-Second-Edition/data/'

In [9]:
movies = pd.read_csv(data + 'movie.csv')
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [6]:
columns = movies.columns
index = movies.index
data = movies.to_numpy()
data

array([['Color', 'James Cameron', 723.0, ..., 7.9, 1.78, 33000],
       ['Color', 'Gore Verbinski', 302.0, ..., 7.1, 2.35, 0],
       ['Color', 'Sam Mendes', 602.0, ..., 6.8, 2.35, 85000],
       ...,
       ['Color', 'Benjamin Roberds', 13.0, ..., 6.3, nan, 16],
       ['Color', 'Daniel Hsia', 14.0, ..., 6.3, 2.35, 660],
       ['Color', 'Jon Gunn', 43.0, ..., 6.6, 1.85, 456]], dtype=object)

In [9]:
type(columns) # pandas.core.indexes.base.Index
type(index)  # pandas.core.indexes.range.RangeIndex
type(data)   # numpy.ndarray

numpy.ndarray

In [10]:
issubclass(pd.RangeIndex, pd.Index) # True == Index --> NumericIndex --> IntegerIndex --> Int64Index -->RangeIndex

True

In [12]:
issubclass(columns.__class__, pd.Index) # True

True

In [13]:
movies.values

array([['Color', 'James Cameron', 723.0, ..., 7.9, 1.78, 33000],
       ['Color', 'Gore Verbinski', 302.0, ..., 7.1, 2.35, 0],
       ['Color', 'Sam Mendes', 602.0, ..., 6.8, 2.35, 85000],
       ...,
       ['Color', 'Benjamin Roberds', 13.0, ..., 6.3, nan, 16],
       ['Color', 'Daniel Hsia', 14.0, ..., 6.3, 2.35, 660],
       ['Color', 'Jon Gunn', 43.0, ..., 6.6, 1.85, 456]], dtype=object)

In [14]:
movies.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

In [27]:
movies.dtypes.value_counts()
movies['director_name'].value_counts()

Steven Spielberg     26
Woody Allen          22
Clint Eastwood       20
Martin Scorsese      20
Ridley Scott         16
                     ..
Fred Durst            1
Jason Connery         1
Vladlen Barbe         1
Floria Sigismondi     1
Lucky McKee           1
Name: director_name, Length: 2397, dtype: int64

In [28]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4897 non-null   object 
 1   director_name              4814 non-null   object 
 2   num_critic_for_reviews     4867 non-null   float64
 3   duration                   4901 non-null   float64
 4   director_facebook_likes    4814 non-null   float64
 5   actor_3_facebook_likes     4893 non-null   float64
 6   actor_2_name               4903 non-null   object 
 7   actor_1_facebook_likes     4909 non-null   float64
 8   gross                      4054 non-null   float64
 9   genres                     4916 non-null   object 
 10  actor_1_name               4909 non-null   object 
 11  movie_title                4916 non-null   object 
 12  num_voted_users            4916 non-null   int64  
 13  cast_total_facebook_likes  4916 non-null   int64

In [34]:
pd.Series(['Paul', np.nan, 'George', 10.]).dtypes # dtype('O')

dtype('O')

In [35]:
movies.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

In [43]:
movies.director_name 

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4             Doug Walker
              ...        
4911          Scott Smith
4912                  NaN
4913     Benjamin Roberds
4914          Daniel Hsia
4915             Jon Gunn
Name: director_name, Length: 4916, dtype: object

In [44]:
movies['director_name']

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4             Doug Walker
              ...        
4911          Scott Smith
4912                  NaN
4913     Benjamin Roberds
4914          Daniel Hsia
4915             Jon Gunn
Name: director_name, Length: 4916, dtype: object

In [41]:
id(movies.director_name.iloc[4912])

4377252848

In [47]:
movies.loc[:, 'director_name']

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4             Doug Walker
              ...        
4911          Scott Smith
4912                  NaN
4913     Benjamin Roberds
4914          Daniel Hsia
4915             Jon Gunn
Name: director_name, Length: 4916, dtype: object

In [48]:
movies.iloc[:, 1]

0           James Cameron
1          Gore Verbinski
2              Sam Mendes
3       Christopher Nolan
4             Doug Walker
              ...        
4911          Scott Smith
4912                  NaN
4913     Benjamin Roberds
4914          Daniel Hsia
4915             Jon Gunn
Name: director_name, Length: 4916, dtype: object

In [49]:
type(movies)

pandas.core.frame.DataFrame

In [55]:
movies['director_name'].index # RangeIndex(start=0, stop=4916, step=1)
movies['director_name'].dtype # dtype('O')
movies['director_name'].size  # 4916
movies['director_name'].name  # 'director_name
type(movies['director_name']) # pandas.core.series.Series

pandas.core.series.Series

In [62]:
movies['director_name'].apply(type).unique() # array([<class 'str'>, <class 'float'>], dtype=object)


array([<class 'str'>, <class 'float'>], dtype=object)

In [71]:
s_attr_methods = set(dir(pd.Series))
len(s_attr_methods) # 434

434

In [72]:
df_attr_methods = set(dir(pd.DataFrame))
len(df_attr_methods) # 441

441

In [73]:
len(s_attr_methods & df_attr_methods) # 384

384

In [75]:
director = movies.director_name
fb_likes = movies.actor_1_facebook_likes
director.dtype # dtype('O')
fb_likes.dtype # dtype('float64')

dtype('float64')

In [76]:
director.head()

0        James Cameron
1       Gore Verbinski
2           Sam Mendes
3    Christopher Nolan
4          Doug Walker
Name: director_name, dtype: object

In [80]:
director.sample(5, random_state=21)

3568     Claude Chabrol
4056      Monte Hellman
254     Martin Scorsese
3576        Greg Marcks
3021       Lisa Azuelos
Name: director_name, dtype: object

In [81]:
director.value_counts()

Steven Spielberg     26
Woody Allen          22
Clint Eastwood       20
Martin Scorsese      20
Ridley Scott         16
                     ..
Fred Durst            1
Jason Connery         1
Vladlen Barbe         1
Floria Sigismondi     1
Lucky McKee           1
Name: director_name, Length: 2397, dtype: int64

In [82]:
fb_likes.value_counts()

1000.0     436
11000.0    206
2000.0     189
3000.0     150
12000.0    131
          ... 
564.0        1
46000.0      1
49.0         1
447.0        1
161.0        1
Name: actor_1_facebook_likes, Length: 877, dtype: int64

In [100]:
director.size # 4916
type(director.size) # int
director.shape # (4916,)
type(director.shape) # tuple
type(director.unique())

numpy.ndarray

In [101]:
director.unique()

array(['James Cameron', 'Gore Verbinski', 'Sam Mendes', ...,
       'Scott Smith', 'Benjamin Roberds', 'Daniel Hsia'], dtype=object)

def decor_upper(func):
    def wrapper():
        orig_func = func()
        modified_ver = func().upper() ## Original function is called here
        return modified_ver
    return wrapper

@decor_upper
def greet():
    return 'hello'
@decor_upper
def abuse():
    return 'bastard'

print(greet()) # HELLO
print(abuse()) # BASTARD

# Book - From Python to Numpy

In [3]:
import random
import timeit
class RandomWalker:
    def __init__(self):
        self.position = 0
        
    def walk(self, n):
        self.position = 0
        for i in range(n):
            yield self.position
            self.position += 2 * random.randint(0,1) - 1

walker = RandomWalker()
timeit.Timer('[position for position in walker.walk(10000)]', 'from __main__ import walker').timeit(100)
# 1.418

1.4564085540000065

In [5]:
def walk(n):
    position = 0
    walk = [position]
    for i in range(n):
        position += 2 * random.randint(0,1) - 1
        walk.append(position)
    return walk
timeit.Timer('walk(10000)', 'from __main__ import walk').timeit(100)

1.274039820000013

In [94]:
def random_walk(n=1000):
    steps = np.random.choice([-1,+1], n)
    return np.cumsum(steps)
timeit.Timer('random_walk(10000)', 'from __main__ import random_walk').timeit(100)

0.01899421199959761

In [92]:
np.cumsum(np.random.choice([-1, +1], 10))

array([-1, -2, -1,  0, -1, -2, -3, -4, -3, -2])

In [3]:
%timeit [i*i for i in range(10)]

945 ns ± 4.84 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [3]:
import line_profiler
%load_ext line_profiler
%lprun -f walk(1000)

In [21]:
df = pd.concat([movies, movies, movies])
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [22]:
def f(row):
    row = row + row
    return np.cumsum(row)

In [23]:
%lprun -f f df.apply(lambda row: f(df['duration']))

In [41]:
a = np.arange(20).reshape(5,4)
print(a.ndim) # 2
print(a.shape) # (5,4)
print(a.size) # 20
print(a.dtype) # int64
print(a.itemsize) # 8, size of each item in a array in byte
print(a.data) # 0x126b50520 - Memory location
print(type(a)) # <class 'numpy.ndarray'>

2
(5, 4)
20
int64
8
<class 'numpy.ndarray'>


In [45]:
a = np.array([[1,1,[2,3]], [2,[2,4],3], [[3,1],3,4]], dtype='object')
a.shape # (3, 3)

(3, 3)

In [47]:
np.empty([2,2])

array([[0.0e+000, 4.9e-324],
       [9.9e-324, 1.5e-323]])

In [49]:
np.zeros([2,2])

array([[0., 0.],
       [0., 0.]])

In [51]:
np.ones((3,3), dtype=np.int16)

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int16)

In [3]:
.6+.8-(.6*.8) 

0.9199999999999999

In [2]:
np.linspace(0,2,9)

array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  ])

In [3]:
np.arange(24).reshape(2,3,4)

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [6]:
np.arange(100000) # array([    0,     1,     2, ..., 99997, 99998, 99999])

array([    0,     1,     2, ..., 99997, 99998, 99999])

In [7]:
np.arange(10000).reshape(100,100)

array([[   0,    1,    2, ...,   97,   98,   99],
       [ 100,  101,  102, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  297,  298,  299],
       ...,
       [9700, 9701, 9702, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9897, 9898, 9899],
       [9900, 9901, 9902, ..., 9997, 9998, 9999]])

In [8]:
a = np.array([[1,1],[0,1]])
b = np.array([[2,0],[3,4]])

In [9]:
a*b

array([[2, 0],
       [0, 4]])

In [10]:
a @ b

array([[5, 4],
       [3, 4]])

In [11]:
a.dot(b)

array([[5, 4],
       [3, 4]])

In [25]:
rg = np.random.default_rng(1) ## create instance of default random number generator
a = np.ones([2,3], dtype=int)
b = rg.random([2,3])
a *= 3
b += a
a += b

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

In [33]:
a = np.ones(3, dtype=np.int32)
b = np.linspace(0, np.pi, 3)
b.dtype.name # 'float64'
a.dtype.name # 'int32'
c = a + b
c.dtype.name # 'float64'
d = np.exp(c*1j)
d.dtype.name # 'complex128'

'complex128'

In [44]:
a = rg.random((2,3))
a.sum() # 2.412
a.max() # 0.980
a.min() # 0.160
a.mean() # 0.593
a.var() # 0.0829
a.std() # 0.329

0.3293018873225993

In [59]:
b = np.arange(6).reshape(2,3)
print(b)
b.sum(axis=1) # sum of each row
b.sum(axis=0) # sum of each column
b.cumsum(axis=0) # cumulative sum of each 

[[0 1 2]
 [3 4 5]]


array([[0, 1, 2],
       [3, 5, 7]])

In [109]:
b = np.arange(3) # array([0, 1, 2])
print(b)
np.exp(b) # array([1.        , 2.71828183, 7.3890561 ])
np.sqrt(b) # array([0.        , 1.        , 1.41421356])
c = np.array([2., -1., 4.])
print('*'*10)
print(c)
print('*'*10)
np.add(b,c) # array([2., 0., 6.])
np.all(b) # False
np.any(b) # True
d = np.array([[9 ,11, 2], [1, 6, 8]])
print(d)

# Apply a function to 1-D slices along the given axis.
np.apply_along_axis(lambda x: (x[0] + x[-1])*.5 , 1, d )  # array([2., 8.])
# np.apply_over_axes  ## Apply the function over multiple axis

np.argmax(d) # 1  -- return the max index
np.argmin(d) # 3  -- return the max index
np.max(d) # 11    -- returns max value

np.argsort(d) ## get the index of the elements as per sorted list

np.average(np.arange(1, 5), weights=np.arange(4)) # 3.33 ## Returns weighted average

# occurence of each element in an array 
np.bincount([1,2,1,2,1,4]) #  array([0, 3, 2, 0, 1])

[0 1 2]
**********
[ 2. -1.  4.]
**********
[[ 9 11  2]
 [ 1  6  8]]


array([0, 3, 2, 0, 1])

In [21]:
x, y = np.ogrid[:3, :4]
x @ y
x.dot(y)

array([[0, 0, 0, 0],
       [0, 1, 2, 3],
       [0, 2, 4, 6]])

In [7]:
px = np.array([19,19.10,21,20.05])
vol = np.array([100,110, 10000, 110000])
np.average(px, weights=vol)

20.1272855835621

In [10]:
np.ceil(4.05) # 5
np.clip(np.arange(10), a_min=1, a_max=8) # array([1, 1, 2, 3, 4, 5, 6, 7, 8, 8])

array([1, 1, 2, 3, 4, 5, 6, 7, 8, 8])

In [11]:
rg = np.random.default_rng(seed=42)
xarr = rg.random((3, 3))
np.corrcoef(xarr)

array([[ 1.        ,  0.99256089, -0.68080986],
       [ 0.99256089,  1.        , -0.76492172],
       [-0.68080986, -0.76492172,  1.        ]])

In [19]:
np.cumsum(np.arange(1,6)) # array([ 1,  3,  6, 10, 15])

array([ 1,  3,  6, 10, 15])

In [20]:
np.diff(np.arange(5)) # array([1, 1, 1, 1])

array([1, 1, 1, 1])

In [70]:
np.inner(np.arange(1,3), np.arange(3,5))

11

In [26]:
np.invert([2]) # array([-3])
np.invert(-1*np.arange(5)) # array([-1, -2, -3, -4, -5])

array([-1,  0,  1,  2,  3])

In [32]:
np.invert([True, False]) # array([False,  True])

array([False,  True])

In [42]:
names = ('Ramesh', 'Suresh', 'Chandu')
surname = ('Ray', 'Cur', 'Shu')
[surname[i] + "," + names[i] for i in np.lexsort((names, surname))]

['Cur,Suresh', 'Ray,Ramesh', 'Shu,Chandu']

In [43]:
np.maximum([1,5,3], [2,3,4]) # array([2, 5, 4])

array([2, 5, 4])

In [45]:
np.mean(np.arange(1,6)) # 3.0
np.median(np.arange(6,1,-1)) # 4.0

4.0

In [46]:
np.maximum([1,5,3], [2,3,4]) # array([2, 5, 4])

array([2, 5, 4])

In [47]:
np.nonzero([0,1,0,1,0,2,0,21.]) # (array([1, 3, 5, 7]),)

(array([1, 3, 5, 7]),)

In [53]:
np.prod(np.arange(1,6))

120

In [56]:
np.round([1.3, 1.5]) # array([1., 2.])

array([1., 2.])

In [57]:
np.sort(np.arange(10,1,-2)) # array([ 2,  4,  6,  8, 10])

array([ 2,  4,  6,  8, 10])

In [59]:
np.std(np.arange(1,5)) # 1.118
np.var(np.arange(1,5)) # 1.25

1.25

In [60]:
np.sum(np.arange(1,11)) # 55

55

In [65]:
np.trace(np.eye(3)) # 3.0
np.trace(np.arange(1,10).reshape(3,3))

15

In [69]:
np.vdot(np.arange(1,5).reshape(2,2), np.arange(1,5).reshape(2,2))

30

In [4]:
np.arange(1,5) * np.arange(2,5)

ValueError: operands could not be broadcast together with shapes (4,) (3,) 

In [5]:
np.arange(9).reshape(3,3) + np.arange(1,4)

array([[ 1,  3,  5],
       [ 4,  6,  8],
       [ 7,  9, 11]])

In [9]:
a = np.array([0, 10, 20, 30])
b = np.array([1., 2., 3.])
a[:, np.newaxis] + b

array([[ 1.,  2.,  3.],
       [11., 12., 13.],
       [21., 22., 23.],
       [31., 32., 33.]])

In [18]:
def myfunc(a,b):
    return a-b if a>b else b-a

vfunc = np.vectorize(myfunc)
vfunc(np.array([100, 90, 80, 70]),10)

array([90, 80, 70, 60])

In [27]:
np.where(10.<20)

(array([0]),)

In [33]:
a = np.arange(10)
np.where(a>3, a, a*10)

array([ 0, 10, 20, 30,  4,  5,  6,  7,  8,  9])

In [46]:
df1 = pd.DataFrame({
    'a':np.arange(10,40,5),
    'b':np.arange(10,16)
})
df1

,a,b
0,10,10
1,15,11
2,20,12
3,25,13
4,30,14
5,35,15


In [54]:
df2 = pd.DataFrame({
    'a':np.arange(10,40,5),
    'b':np.array([20,21,np.nan,23,24,np.nan])
})
df2

,a,b
0,10,20.0
1,15,21.0
2,20,NaN
3,25,23.0
4,30,24.0
5,35,NaN


In [56]:
pd.merge(df1, df2, on='b', how='right')

,a_x,b,a_y
0,NaN,20.0,10
1,NaN,21.0,15
2,NaN,NaN,20
3,NaN,23.0,25
4,NaN,24.0,30
5,NaN,NaN,35


In [4]:
a = np.arange(10)**3
a # array([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729])

array([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729])

In [5]:
a[2]

8

In [6]:
a[:6:2]

array([ 0,  8, 64])

In [8]:
np.fromfunction(lambda i,j: i+j, (3,3), dtype=int)

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

In [9]:
np.fromfunction(lambda i,j: i==j, (3,3), dtype=int)

array([[ True, False, False],
       [False,  True, False],
       [False, False,  True]])

In [15]:
b = np.fromfunction(lambda i,j:10*i+j, (5,4), dtype=int)
print(b)
b[2,3]
b[0:4, 1] # # each row in the second column of b


[[ 0  1  2  3]
 [10 11 12 13]
 [20 21 22 23]
 [30 31 32 33]
 [40 41 42 43]]


array([ 1, 11, 21, 31])

In [16]:
b[-1] # array([40, 41, 42, 43])

array([40, 41, 42, 43])

In [17]:
b[1,...] # array([10, 11, 12, 13])

array([10, 11, 12, 13])

In [19]:
for row in b:
    print(row*10)

[ 0 10 20 30]
[100 110 120 130]
[200 210 220 230]
[300 310 320 330]
[400 410 420 430]


In [37]:
rg = np.random.default_rng(seed=42)
a = np.floor(10*rg.random((3,4)))
a.shape

(3, 4)

In [28]:
a.ravel() # array([7., 4., 8., 6., 0., 9., 7., 7., 1., 4., 3., 9.])

array([7., 4., 8., 6., 0., 9., 7., 7., 1., 4., 3., 9.])

In [30]:
a.reshape(2,6)

array([[7., 4., 8., 6., 0., 9.],
       [7., 7., 1., 4., 3., 9.]])

In [31]:
a.T

array([[7., 0., 1.],
       [4., 9., 4.],
       [8., 7., 3.],
       [6., 7., 9.]])

In [39]:
a.shape # (3, 4)
a.resize(4,3)
a.shape # (4, 3)

(4, 3)

In [43]:
a.reshape(6,-1)

array([[7., 4.],
       [8., 6.],
       [0., 9.],
       [7., 7.],
       [1., 4.],
       [3., 9.]])

In [47]:
a = np.arange(4).reshape(2,-1)
b = np.arange(5,9).reshape(2,-1)
np.vstack((a,b))

array([[0, 1],
       [2, 3],
       [5, 6],
       [7, 8]])

In [48]:
np.hstack((a,b))

array([[0, 1, 5, 6],
       [2, 3, 7, 8]])

In [59]:
a = np.arange(5)
b = np.arange(5,10)
np.column_stack((a,b))

array([[0, 5],
       [1, 6],
       [2, 7],
       [3, 8],
       [4, 9]])

In [60]:
np.row_stack is np.vstack

True

In [63]:
np.r_[1:4,11,22] # array([ 1,  2,  3, 11, 22])
np.c_[1:4,1:4]

array([[1, 1],
       [2, 2],
       [3, 3]])

In [72]:
a = np.arange(16).reshape(4,-1)
np.hsplit(a, 2)
np.vsplit(a,2)

[array([[0, 1, 2, 3],
        [4, 5, 6, 7]]),
 array([[ 8,  9, 10, 11],
        [12, 13, 14, 15]])]

In [124]:
df1 = pd.DataFrame({
    'key':['z','a','b','c'],
    'val':[100,10, 20, np.nan]
})
df2 = pd.DataFrame({
    'key':['a','b','c','d','e'],
    'val':[10, 20, 30,40,50]
})
print(df1)
print('*'*10)
print(df2)

  key    val
0   z  100.0
1   a   10.0
2   b   20.0
3   c    NaN
**********
  key  val
0   a   10
1   b   20
2   c   30
3   d   40
4   e   50


In [132]:
df3 = df1.set_index('key').join(df2.set_index('key'), lsuffix='_x', rsuffix='_y', how='outer').reset_index()
df3['val'] = df3.val_x.fillna(df3.val_y)
df3 = df3.drop(['val_x','val_y'], axis=1).rename(columns={'val_y':'val'})
df3

,key,val
0,a,10.0
1,b,20.0
2,c,30.0
3,d,40.0
4,e,50.0
5,z,100.0


In [138]:
df3 = df1.merge(df2, on='key', how='outer')
df3['val'] = df3.val_x.fillna(df3.val_y)
df3 = df3.drop(['val_x','val_y'], axis=1).rename(columns={'val_y':'val'})
df3.drop(['key'], axis=1, inplace=True)
df3

,val
0,100.0
1,10.0
2,20.0
3,30.0
4,40.0
5,50.0


In [26]:
a = np.arange(5)
b = a
print(id(a)) # 4902824816
print(id(b)) # 4902824816
print(a is b) # True
print(a == b) # [ True  True  True  True  True]

4902824816
4902824816
True
[ True  True  True  True  True]


In [9]:
def f(x):
    print(id(x))

t = (1,2,3,4)
print(id(t)) # 4902145936
f(t) # 4902145936

4902145936
4902145936


In [25]:
a = np.array([1,2,3])
print(a.base) # None
c = a.view()

print(id(a.base)) # 4527528432
print(id(a)) #  4902824016

print(id(c.base)) # 4902824016
print(id(c)) # 4902822896



None
4527528432
4902825216
4902825216
4902825376


In [21]:
b = np.arange(2)
b.base is None

True

In [39]:
a = np.arange(6)
print(a.flags.owndata) # True
c = a.view()
print(id(a)) #4902825456
print(id(c)) #4932942384
print(id(c.base)) # 4932943184

print (c is a) # False
print (c.base is a) # True

#np.flags - Information about the memory layout of the array.
print(c.flags.owndata) # False

c = c.reshape(2,-1)
print(c.shape) # (2, 3)

print(a.shape) # (6,)

c[0,0] = 1234
print(c[0,0]) # 1234

print(a) # [1234    1    2    3    4    5]

True
4921765680
4922930480
4921765680
False
True
False
(2, 3)
(6,)
1234
[1234    1    2    3    4    5]


In [41]:
s = a[:]
print(s) # [1234    1    2    3    4    5]
print(a) # [1234    1    2    3    4    5]
s[:] = 10
print(s) # [10 10 10 10 10 10]
print(a) # [10 10 10 10 10 10]

[1234    1    2    3    4    5]
[1234    1    2    3    4    5]
[10 10 10 10 10 10]
[10 10 10 10 10 10]


In [47]:
a = np.arange(4)
print(a.base) # None
print(id(a.base)) # 4527528432
print(id(a)) # 4932945360

d = a.copy()
print(d.base) # None
print(id(d.base)) # 4527528432
print(id(d)) # 4920218768

d.base is a  # False - d doesn't share anything with a

d[2] = 10
print(d) # [ 0  1 10  3]
print(a) # [0 1 2 3]

None
4527528432
4921762640
None
4527528432
4921764800
[ 0  1 10  3]
[0 1 2 3]


In [53]:
a = np.arange(1e3)
print(id(a)) # 4932947600
b = a[:100].copy()
print(id(b.base)) # 4527528432
del a # The memory of a can be released

4932947600
4527528432


In [56]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [60]:
a = np.empty(6).reshape(2,3)

array([[0.000e+00, 1.000e+00, 2.000e+00],
       [9.997e+03, 9.998e+03, 9.999e+03]])

In [61]:
a = np.empty(6).reshape(2,3)
print(a)
b = np.empty_like(a)
print(b)

[[0.000e+00 1.000e+00 2.000e+00]
 [9.997e+03 9.998e+03 9.999e+03]]
[[0.000e+00 1.000e+00 2.000e+00]
 [9.997e+03 9.998e+03 9.999e+03]]


In [63]:
np.fromfunction(lambda i,j: i*j, (3,3), dtype=int)

array([[0, 0, 0],
       [0, 1, 2],
       [0, 2, 4]])

In [68]:
np.eye(2,3)

array([[1., 0., 0.],
       [0., 1., 0.]])

In [72]:
np.identity(2)

array([[1., 0.],
       [0., 1.]])

In [83]:
%timeit np.linspace(2,200,10) # 32.7 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
%timeit np.arange(2,201,2) # 935 ns ± 26.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
%timeit [i for i in range(2,201,2)]

33.7 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
932 ns ± 3.28 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
4.09 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [84]:
np.logspace(1,20,10)

array([1.00000000e+01, 1.29154967e+03, 1.66810054e+05, 2.15443469e+07,
       2.78255940e+09, 3.59381366e+11, 4.64158883e+13, 5.99484250e+15,
       7.74263683e+17, 1.00000000e+20])

In [95]:
np.mgrid[:3,:2]

array([[[0, 0],
        [1, 1],
        [2, 2]],

       [[0, 1],
        [0, 1],
        [0, 1]]])

In [96]:
x,y = np.ogrid[:3, :4]
x@y

array([[0, 0, 0, 0],
       [0, 1, 2, 3],
       [0, 2, 4, 6]])

In [99]:
np.ones([2,3])

array([[1., 1., 1.],
       [1., 1., 1.]])

In [102]:
a = np.arange(6).reshape(2,-1)
np.ones_like(a)

array([[1, 1, 1],
       [1, 1, 1]])

In [5]:
np.r_[([1,2,3,4,5,6,7][1:5]),[3,4]]

array([2, 3, 4, 5, 3, 4])

In [7]:
a = np.arange(6)
print(a.dtype)
a = a.astype(float)
print(a.dtype)

int64
float64


In [18]:
np.atleast_3d(1, [2,4], [[1,2],[3,4]])

[array([[[1]]]),
 array([[[2],
         [4]]]),
 array([[[1],
         [2]],
 
        [[3],
         [4]]])]

In [21]:
np.asmatrix(np.arange(4).reshape(2,-1))

matrix([[0, 1],
        [2, 3]])

In [67]:
df = pd.DataFrame({
    'exp':['1y','1y','1y','1y','1y','1y','1y','1y','1y',],
    'mat':['1y','1y','1y','2y','2y','2y','1y','1y','1y',],
    'reg':['in', 'in', 'in', 'in', 'in', 'in','in','in','in'],
    'con':['5w','5s','5c','5w','5s','5c','10w','10s','10c'],
    'pcon':['w','s','c','w','s','c','w','s','c'],
    'val':[2.5,0,-2.5,2.5,0,-2.5,5,0,-5],
    'rs':[6, 10, 4, 12, 30, 6,30,20,10]
})
df

,exp,mat,reg,con,pcon,val,rs
0,1y,1y,in,5w,w,2.5,6
1,1y,1y,in,5s,s,0.0,10
2,1y,1y,in,5c,c,-2.5,4
3,1y,2y,in,5w,w,2.5,12
4,1y,2y,in,5s,s,0.0,30
5,1y,2y,in,5c,c,-2.5,6
6,1y,1y,in,10w,w,5.0,30
7,1y,1y,in,10s,s,0.0,20
8,1y,1y,in,10c,c,-5.0,10


In [69]:
df = pd.DataFrame({
    'exp':['1y','1y','1y','1y','1y','1y','1y','1y','1y',],
    'mat':['1y','1y','1y','2y','2y','2y','1y','1y','1y',],
    'reg':['in', 'in', 'in', 'in', 'in', 'in','in','in','in'],
    'con':['5w','5s','5c','5w','5s','5c','10w','10s','10c'],
    'pcon':['w','s','c','w','s','c','w','s','c'],
    'val':[2.5,0,-2.5,2.5,0,-2.5,5,0,-5],
    'rs':[5, 10, 1, 9, 30, 3 ,20,25,10]
})
df

,exp,mat,reg,con,pcon,val,rs
0,1y,1y,in,5w,w,2.5,5
1,1y,1y,in,5s,s,0.0,10
2,1y,1y,in,5c,c,-2.5,1
3,1y,2y,in,5w,w,2.5,9
4,1y,2y,in,5s,s,0.0,30
5,1y,2y,in,5c,c,-2.5,3
6,1y,1y,in,10w,w,5.0,20
7,1y,1y,in,10s,s,0.0,25
8,1y,1y,in,10c,c,-5.0,10


In [75]:
tmp = df.set_index(['exp','mat','reg', 'con']).unstack() ## Failing here on unstack
tmp[('rs','w')],tmp[('rs','c')] = ((tmp[('rs','w')] + tmp[('rs','c')])/2,
                                  (tmp[('rs','w')] - tmp[('rs','c')])/2)
tmp.stack().reset_index()

KeyError: ('rs', 'w')

In [51]:
df = pd.pivot_table(df, index=['exp','mat','reg', 'con'], values=['rs'], 
               aggfunc=np.sum
              )
df

rs
exp mat reg con    
1y  1y  in  c     4
            s    10
            w     6
    2y  in  c     6
            s    30
            w    12

In [23]:
a = np.arange(5)
b = np.arange(5)

In [24]:
a

array([0, 1, 2, 3, 4])

In [25]:
b

array([0, 1, 2, 3, 4])

In [28]:
np.stack((a,b), axis=1)

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4]])

In [27]:
np.vstack((a,b))

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [29]:
np.dstack((a,b))

array([[[0, 0],
        [1, 1],
        [2, 2],
        [3, 3],
        [4, 4]]])

In [45]:
a = np.arange(16).reshape(2,2,4)

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]]])

In [46]:
np.dsplit(a,2)

[array([[[ 0,  1],
         [ 4,  5]],
 
        [[ 8,  9],
         [12, 13]]]),
 array([[[ 2,  3],
         [ 6,  7]],
 
        [[10, 11],
         [14, 15]]])]

In [68]:
a = np.arange(6)

In [69]:
np.split(a, 4)

ValueError: array split does not result in an equal division

In [67]:
np.array_split(a, 3)

[array([0]), array([1]), array([], dtype=int64)]

In [70]:
a = np.arange(5)
b = np.arange(5,10)
np.concatenate((a,b))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [78]:
np.vstack((a,b)).ravel()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [81]:
x = np.array([1,2,3,-1,5])
print(x.mean()) # 2.0 wrong output because of invalid entry in dataset
mx = ma.masked_array(x, mask=[0,0,0,1,0])
mx.mean() # 2.75 correct ouput after masking x as mx

2.0


2.75

In [83]:
ma.masked_values([1.0,1.e20, 3.0,4.0], 1.e20).sum()

8.0

In [88]:
a = np.array([10,20,30])
ma.array(a, mask=[0,1,0])

masked_array(data=[10, --, 30],
             mask=[False,  True, False],
       fill_value=999999)

In [116]:
a = np.arange(10)
m = ma.masked_less_equal(a, 5)
m[~m.mask]
# masked_array(data=[6, 7, 8, 9],
#             mask=[False, False, False, False],
#       fill_value=999999)

m.compressed() # array([6, 7, 8, 9])

array([6, 7, 8, 9])

In [123]:
x = np.array([1, 2, 3])
m = ma.array(x)
m[0] = ma.masked
m

masked_array(data=[--, 2, 3],
             mask=[ True, False, False],
       fill_value=999999)

In [139]:
y = ma.masked_array(np.arange(9).reshape(3,-1))
y[(0,1,2),(1,2,0)] = ma.masked
y

masked_array(
  data=[[0, --, 2],
        [3, 4, --],
        [--, 7, 8]],
  mask=[[False,  True, False],
        [False, False,  True],
        [ True, False, False]],
  fill_value=999999)

In [133]:
np.eye(3,3).astype(int)

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [134]:
np.arange(9).reshape(3,-1)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [144]:
a = ma.array(np.arange(4), mask=[0,0,1,0], hard_mask=True)
a
a[-2] = 20 ## Nothing changed after this

a.soften_mask()
a[-2] = 20
a  ## value of a[-2] has been changed to 20

a.harden_mask()

masked_array(data=[0, 1, 20, 3],
             mask=[False, False, False, False],
       fill_value=999999)

In [146]:
a = ma.array(np.arange(4), mask=[0,0,1,0])
a.mask = ma.nomask
a

masked_array(data=[0, 1, 2, 3],
             mask=[False, False, False, False],
       fill_value=999999)

In [149]:
a = ma.array(np.arange(4), mask=[0,0,1,0])
a[1] # 1
a[2] # masked

a[2] is ma.masked ## True

True

In [162]:
a = ma.array(np.arange(4).reshape(2,-1), mask=[(0,0), (0,1)])
a[0].data # data=[0, 1]
a[1] # data=[2, --]
a[1,1] # masked

masked

In [166]:
a = ma.array(np.arange(1,6), mask=[0,1,0,0,1])
a[1:3] # data=[--, 3],
a.data # array([1, 2, 3, 4, 5])
a.mask # array([False,  True, False, False,  True])
a.compressed() # array([1, 3, 4])

array([1, 3, 4])

In [ ]:
a = ma.array(ma.arange(1,5))

In [184]:
a = ma.concatenate((ma.arange(1,5), ma.array([999])))
a[-1] = ma.masked
a.mean()
print(a-a.mean())
print(a)
print(a.filled(a.mean()))
print(a)

[-1.5 -0.5 0.5 1.5 --]
[1 2 3 4 --]
[1 2 3 4 2]
[1 2 3 4 --]


In [228]:
d = np.arange(10)
d.mean() # 4.5
ma.masked_outside(d,5,8).mean() # 6.5

6.5

In [229]:
m = (d>4) & (d<9)
np.where(m,d,0).mean()

2.6

In [236]:
z = [['a','b','c'],['d','e','f'],['g','h','i']]


In [237]:
%timeit list(zip(*z))[1]

572 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [239]:
%timeit l = []; [l.append(i[1]) for i in a]

621 ns ± 5.72 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [38]:
aa=['h1']
bb=['b1','b2']
cc=['c1','c2','c3']
dd=['d1','d2','d3','d4','d5']
df = pd.DataFrame()
#df.insert(1,'b', [b])
#df.insert(2,'c', [c])
#df.insert(3,'d', [d])
#df.explode('b').explode('c').explode('d')
df = df.assign(a=[aa], b=[bb], c=[cc], d=[dd])

z = ['z1']
y = ['y1']
df1=pd.DataFrame()
df1 = df1.assign(z=z, y=y)
df1 = pd.concat([df1, df], axis=1)
df1 = df1.explode('y').explode('a').explode('b').explode('c').explode('d')
df1

,z,y,a,b,c,d
0,z1,y1,h1,b1,c1,d1
0,z1,y1,h1,b1,c1,d2
0,z1,y1,h1,b1,c1,d3
0,z1,y1,h1,b1,c1,d4
0,z1,y1,h1,b1,c1,d5
0,z1,y1,h1,b1,c2,d1
0,z1,y1,h1,b1,c2,d2
0,z1,y1,h1,b1,c2,d3
0,z1,y1,h1,b1,c2,d4
0,z1,y1,h1,b1,c2,d5


In [4]:
a = np.arange(5)
b = ma.array([10,11,12], mask=[0,1,0])
np.concatenate([a,b])

masked_array(data=[ 0,  1,  2,  3,  4, 10, 11, 12],
             mask=False,
       fill_value=999999)

In [29]:
a = np.arange(4).reshape(2,2)
print(a)
np.fliplr(a).diagonal() # array([1, 2]) # Horizontal flip
np.flipud(a).diagonal() # array([2, 1]) # Vertical flip

[[0 1]
 [2 3]]


array([2, 1])

In [33]:
df=DataFrame({
    'a':[1,2],
    'b':[3,4]
})
np.diagonal(df)

array([1, 4])

In [43]:
np.random.seed(3)
x = np.random.randint(9, size=(3,3))
print(x)
x.item((2,1)) # 5

[[8 3 8]
 [8 0 5]
 [3 5 7]]


5

In [54]:
x = np.array([1,2,3])
x[:, np.newaxis]
x[:, np.newaxis] * x

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [57]:
df = DataFrame({
    'a':[10,20],
    'b':[30,40],
    'c':[50,60]
})
df.melt(id_vars=['a'], var_name='d', value_name='e')

,a,d,e
0,10,b,30
1,20,b,40
2,10,c,50
3,20,c,60


In [66]:
a = np.arange(4).reshape(2,-1)
print(a)
np.flip(a)

[[0 1]
 [2 3]]


array([[3, 2],
       [1, 0]])

In [67]:
np.diag([2,2])

array([[2, 0],
       [0, 2]])

In [70]:
(np.eye(2)*2).astype(int)

array([[2, 0],
       [0, 2]])

In [71]:
np.repeat(3,4)

array([3, 3, 3, 3])

In [75]:
np.repeat(np.arange(4).reshape(2,-1), 2, axis=1)

array([[0, 0, 1, 1],
       [2, 2, 3, 3]])

In [97]:
np.repeat(np.arange(4).reshape(2,-1), [2,3], axis=0)


array([[0, 1],
       [0, 1],
       [2, 3],
       [2, 3],
       [2, 3]])

In [105]:
a = np.arange(4)
np.resize(a,3) # array([0, 1, 2])
a= np.arange(6).reshape(2,-1)
print(a)
np.resize(a,[2,5])

[[0 1 2]
 [3 4 5]]


array([[0, 1, 2, 3, 4],
       [5, 0, 1, 2, 3]])

In [18]:
a = np.arange(8).reshape(2,2,-1)
print(a)
print(np.swapaxes(a,0,2))
np.transpose(a)

[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]
[[[0 4]
  [2 6]]

 [[1 5]
  [3 7]]]


array([[[0, 4],
        [2, 6]],

       [[1, 5],
        [3, 7]]])

In [34]:
a = np.array([[1234],[12]])
print(a)
print(np.squeeze(a)[()])
a.ravel()

[[1234]
 [  12]]
[1234   12]


array([1234,   12])

In [39]:
a = np.arange(4).reshape(2,-1)
print(a)
a.transpose()

[[0 1]
 [2 3]]


array([[0, 2],
       [1, 3]])

In [45]:
a = np.arange(4).reshape(2,-1)
indices = [0,2]
np.take(a, indices)

[0 2]


In [49]:
a = np.arange(4)
print(np.any(a)) # True 
# Can be used with ndarray as well
a.any() # True

True


True

In [62]:
a = np.array([0,1,0,2,0,3])
np.nonzero(a) # (array([1, 3, 5]),) -- Indices of nonzero elements
a[np.nonzero(a)] # array([1, 2, 3])

array([1, 2, 3])

In [68]:
a = np.arange(4).reshape(2,-1)
print(a)
np.nonzero(a)

[[0 1]
 [2 3]]


(array([0, 1, 1]), array([1, 0, 1]))

In [76]:
a = np.arange(9).reshape(3,-1)
print(a>3)
np.nonzero(a > 3)
#Using this result to index a is equivalent to using the mask directly:
print(a[np.nonzero(a > 3)])
print(a[a > 3])  # prefer this spelling
# nonzero can also be called as a method of the array.
(a > 3).nonzero()

[[False False False]
 [False  True  True]
 [ True  True  True]]
[4 5 6 7 8]
[4 5 6 7 8]


(array([1, 1, 2, 2, 2]), array([1, 2, 0, 1, 2]))

In [79]:
a = np.arange(8)
np.where(a>5) # DON'T USE, output - (array([6, 7]),)
np.asrray(a>5).nonzero() ### USE THIS, output - (array([6, 7]),)

(array([6, 7]),)

In [85]:
a = np.arange(8)
np.where(a>3, a, a*10) # array([ 0, 10, 20, 30,  4,  5,  6,  7])

array([ 0, 10, 20, 30,  4,  5,  6,  7])

In [86]:
a = np.arange(9).reshape(3,-1)
np.where(a<4, a*20, a)

array([[ 0, 20, 40],
       [60,  4,  5],
       [ 6,  7,  8]])

In [102]:
a = np.random.choice(10, size=8, replace=False).reshape(2,4)
print(a)
print(np.argmax(a))
print(np.argmax(a, axis=0))
print(np.argmax(a, axis=1))

[[9 5 0 8]
 [2 6 4 3]]
0
[0 1 1 0]
[0 1]


In [95]:
np.random.choice(10, size=10, replace=False)

array([2, 6, 0, 5, 7, 1, 8, 3, 4, 9])

In [109]:
np.arange(10,16).reshape(2,3)[np.unravel_index([1, 3, 5], (2,3))]

array([11, 13, 15])

In [113]:
a = np.array([1,3,2,4])
print(np.argsort(a)) # Sorted array Index - [0 2 1 3]
a[np.argsort(a)] # array([1, 2, 3, 4])

[0 2 1 3]


array([1, 2, 3, 4])

In [119]:
a = np.arange(9).reshape(3,3)
np.ptp(a, axis=0)

array([6, 6, 6])

In [126]:
a = np.array([0,10,30,40])
np.searchsorted(a, 20) # 2
a[np.searchsorted(a, 2)] = 20
a

array([ 0, 20, 30, 40])

In [123]:
a = np.array([1,5,3,7])
np.sort(a) # array([1, 3, 5, 7])

array([1, 3, 5, 7])

In [143]:
a = np.arange(9).reshape(3,3)
a.compress([1,2], axis=1)

a = np.arange(9)
a.compress([1,2]) # array([0, 1])
np.extract(a>3, a) # array([4, 5, 6, 7, 8])

array([4, 5, 6, 7, 8])

In [144]:
a = np.arange(4)
b = np.arange(4)
np.inner(a,b) # 14

14

In [147]:
a = np.arange(6).reshape(2,3)
a.flat[4] # 4
a.ravel()[4] # 4

4

In [149]:
a = np.arange(5)
np.put(a, [0,2], [100, 200])
a # array([100,   1, 200,   3,   4])

array([100,   1, 200,   3,   4])

In [151]:
a = np.arange(6).reshape(2,3)
np.putmask(a,a>2,a**2)
a 

array([[ 0,  1,  2],
       [ 9, 16, 25]])

In [152]:
x = np.arange(5)
y = np.arange(5)
np.cov(x,y)

array([[2.5, 2.5],
       [2.5, 2.5]])

In [161]:
a = np.arange(3)
b = np.arange(3)
np.outer(a,b)

array([[0, 0, 0],
       [0, 1, 2],
       [0, 2, 4]])

In [164]:
a = np.arange(8)
i = [1,3]
a[i] # array([1, 3])
j = np.array([[1,3],[2,5]])
a[j]

array([[1, 3],
       [2, 5]])

In [170]:
time = np.linspace(20, 145, 5)
data = np.sin(np.arange(20)).reshape(5,4)
ind = data.argmax(axis=0)
time_max = time[ind]
data_max = data[ind, range(data.shape[1])] 
np.all(data_max == data.max(axis=0))

True

In [177]:
df = pd.DataFrame({
    'exp':['1y','1y','1y','1y','1y','1y','1y','1y','1y',],
    'mat':['1y','1y','1y','2y','2y','2y','1y','1y','1y',],
    'reg':['in', 'in', 'in', 'in', 'in', 'in','in','in','in'],
    'con':['5w','5s','5c','5w','5s','5c','10w','10s','10c'],
    'pcon':['w','s','c','w','s','c','w','s','c'],
    'val':[2.5,0,-2.5,2.5,0,-2.5,5,0,-5],
    'rs':[6, 10, 4, 12, 30, 6,30,25,10]
})
df['tem'] = df['con'].str.extract('(\d+)')
tmp = df.set_index(['exp','mat','reg', 'tem', 'pcon']).unstack() ## Failing here
tmp[('rs','w')],tmp[('rs','c')] = ((tmp[('rs','w')] + tmp[('rs','c')])/2,
                                  (tmp[('rs','w')] - tmp[('rs','c')])/2)
tmp = tmp.stack().reset_index()
tmp

,exp,mat,reg,tem,pcon,con,val,rs
0,1y,1y,in,10,c,10c,-5.0,10.0
1,1y,1y,in,10,s,10s,0.0,25.0
2,1y,1y,in,10,w,10w,5.0,20.0
3,1y,1y,in,5,c,5c,-2.5,1.0
4,1y,1y,in,5,s,5s,0.0,10.0
5,1y,1y,in,5,w,5w,2.5,5.0
6,1y,2y,in,5,c,5c,-2.5,3.0
7,1y,2y,in,5,s,5s,0.0,30.0
8,1y,2y,in,5,w,5w,2.5,9.0
